<a href="https://colab.research.google.com/github/mmmovania/CUDA_Spring2022_GoogleColabs/blob/main/Week8/PrefixSum_WorkEfficient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%cd /usr/local/
!rm -rf cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda
!stat cuda
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

/usr/local
  File: cuda -> /usr/local/cuda-10.1
  Size: 20        	Blocks: 0          IO Block: 4096   symbolic link
Device: 24h/36d	Inode: 4063235     Links: 1
Access: (0777/lrwxrwxrwx)  Uid: (    0/    root)   Gid: (    0/    root)
Access: 2022-03-17 16:07:09.729159579 +0000
Modify: 2022-03-17 16:07:09.625159735 +0000
Change: 2022-03-17 16:07:09.625159735 +0000
 Birth: -
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-1ja_xz7o
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-1ja_xz7o
directory /usr/local/src already exists
Out bin /usr/local/result.out


In [49]:
%%cu
#include <stdio.h>

#define SECTION_SIZE 4 

inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}

//CPU version
void sequential_scan(int* x, int* y, int N) 
{
  y[0]=x[0]; 
  for (int i=1; i < N; i++)
  {
    y[i]= y [i-1] + x[i]; 
  }
}

//Inefficient exclusive scan kernel
__global__ void work_inefficient_exc_scan_kernel(int *X, int *Y, int N) {
  __shared__ int XY[SECTION_SIZE]; 
  int i = blockIdx.x*blockDim.x + threadIdx.x; 
  if (i < N && threadIdx.x != 0) 
  {
    XY[ threadIdx.x ] = X[i-1];
  } 
  else 
  {
    XY[ threadIdx.x ] = 0;
  }

  // the code below performs iterative scan on XY 
  for (unsigned int stride=1; stride<=threadIdx.x; stride*= 2) 
  { 
    __syncthreads(); 
    XY[threadIdx.x] += XY[threadIdx.x-stride]; 
  }
  
  Y[i] = XY[threadIdx.x];
}

//Efficient exclusive scan kernel
__global__ void work_efficient_exc_scan_kernel(int *X, int *Y, int N) {
  __shared__ int XY[SECTION_SIZE]; 
  int i = blockIdx.x*blockDim.x + threadIdx.x; 
  if (i < N && threadIdx.x != 0) 
  {
    XY[ threadIdx.x ] = X[i-1];
  } 
  else 
  {
    XY[ threadIdx.x ] = 0;
  }

  // the code below performs iterative scan on XY 
  for (unsigned int stride=1; stride< blockDim.x; stride*= 2) 
  { 
    __syncthreads(); 
    int index = (threadIdx.x + 1)*2*stride - 1;
    if(index < blockDim.x )
      XY[index] += XY[index-stride]; 
  }

  for (int stride = SECTION_SIZE/2; stride>0; stride/=2)
  {
    __syncthreads();
    int index = (threadIdx.x + 1)*2*stride - 1;
    if(index + stride < 2*blockDim.x)
      XY[index + stride] += XY[index]; 
  }
   
  __syncthreads();
  Y[i] = XY[threadIdx.x];
}

//Inefficient inclusive scan kernel
__global__ void work_inefficient_inc_scan_kernel(int *X, int *Y, int N) {
  __shared__ int XY[SECTION_SIZE]; 
  int i = blockIdx.x*blockDim.x + threadIdx.x; 
  if (i < N) { 
    XY[threadIdx.x] = X[i]; 
  }

  // the code below performs iterative scan on XY 
  for (unsigned int stride=1; stride<=threadIdx.x; stride*= 2) 
  { 
    __syncthreads(); 
    XY[threadIdx.x] += XY[threadIdx.x-stride]; 
  }
  
  Y[i] = XY[threadIdx.x];
}
   
//Efficient inclusive scan kernel
__global__ void work_efficient_inc_scan_kernel(int *X, int *Y, int N) {
  __shared__ int XY[SECTION_SIZE]; 
  int i = blockIdx.x*blockDim.x + threadIdx.x; 
  if (i < N) { 
    XY[threadIdx.x] = X[i]; 
  }

  // the code below performs iterative scan on XY 
  for (unsigned int stride=1; stride< blockDim.x; stride*= 2) 
  { 
    __syncthreads(); 
    int index = (threadIdx.x + 1)*2*stride - 1;
    if(index < blockDim.x)
      XY[index] += XY[index-stride]; 
  }
 
  /*for (int stride = SECTION_SIZE/4; stride>0; stride/=2)
  {
    __syncthreads();
    int index = (threadIdx.x + 1) *2*stride - 1;
    if(index + stride < blockDim.x)
      XY[index + stride] += XY[index]; 
  }*/
  for (int stride = SECTION_SIZE/2; stride>0; stride/=2)
  {
    __syncthreads();
    int index = (threadIdx.x + 1)*2*stride - 1;
    if(index + stride < 2*blockDim.x)
      XY[index + stride] += XY[index]; 
  }
  
  __syncthreads();
  Y[i] = XY[threadIdx.x];
}

int main() 
{ 
		int   *X, *Y; 
    const int N = 32; 
    const int threadsPerBlock = SECTION_SIZE;
    const int blocksPerGrid =  (N / threadsPerBlock);

    // Allocate Unified Memory -- accessible from CPU or GPU
    checkCudaErr(cudaMallocManaged(&X, N*sizeof(int)), "cudaMallocManaged1");
    checkCudaErr(cudaMallocManaged(&Y, N*sizeof(int)), "cudaMallocManaged2"); 
     
    // fill in the memory with data
    for (int i=0; i<N; i++) 
    {
        X[i] = i+1;
        Y[i] = 0; 
    }  
  
    // Prefetch the data to the GPU
    int device = -1;
    cudaGetDevice(&device);
    cudaMemPrefetchAsync(X, N*sizeof(int), device, NULL);
    cudaMemPrefetchAsync(Y, N*sizeof(int), device, NULL);
    
    cudaEvent_t start, stop;
    float cpu_elapsed_time_ms=0,  
          gpu1_elapsed_time_ms=0, 
          gpu2_elapsed_time_ms=0,
          gpu3_elapsed_time_ms=0,
          gpu4_elapsed_time_ms=0;
 
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
 
    ////lets time the CPU code
    cudaEventRecord(start, 0);
      sequential_scan(X, Y, N);
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&cpu_elapsed_time_ms, start, stop);

    if(N<10) {
      puts("Results on CPU: ");
      puts("X[i]\t| Y[i]");
      puts("--------+-------");

      for(int i=0; i<N; ++i)
        printf("%3d\t| %3d\n", X[i], Y[i]);
    }
 
    //reset Y for GPU
    for (int i=0; i<N; i++) 
    {         
        Y[i] = 0; 
    }  

    //lets time the inefficient inclusive scan GPU code  
    cudaEventRecord(start, 0); 
      work_inefficient_inc_scan_kernel<<<1,N>>>(X, Y, N);    
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop); 
    cudaEventElapsedTime(&gpu1_elapsed_time_ms, start, stop);

    if(N<10) {
      puts("Results on GPU (inclusive inefficient): ");
      puts("X[i]\t| Y[i]");
      puts("--------+-------");

      for(int i=0; i<N; ++i)
        printf("%3d\t| %3d\n", X[i], Y[i]);
    }
 
    //reset Y for GPU
    for (int i=0; i<N; i++) 
    {         
        Y[i] = 0; 
    }  
 
    //lets time the inefficient exclusive scan GPU code  
    cudaEventRecord(start, 0);
      work_inefficient_exc_scan_kernel<<<1,N>>>(X, Y, N);    
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop); 
    cudaEventElapsedTime(&gpu2_elapsed_time_ms, start, stop);
 
    if(N<10) {
      puts("----------------");
      puts("Results on GPU (exclusive inefficient): ");
      puts("X[i]\t| Y[i]");
      puts("--------+-------");

      for(int i=0; i<N; ++i)
        printf("%3d\t| %3d\n", X[i], Y[i]);
    }

    //reset Y for GPU
    for (int i=0; i<N; i++) 
    {         
        Y[i] = 0; 
    }  
 
    //lets time the efficient inclusive scan GPU code  
    cudaEventRecord(start, 0);
      work_inefficient_inc_scan_kernel<<<1,N>>>(X, Y, N);    
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop); 
    cudaEventElapsedTime(&gpu3_elapsed_time_ms, start, stop);

    if(N<10) {
      puts("----------------");
      puts("Results on GPU (inclusive efficient): ");
      puts("X[i]\t| Y[i]");
      puts("--------+-------");

      for(int i=0; i<N; ++i)
        printf("%3d\t| %3d\n", X[i], Y[i]);
    }

    //reset Y for GPU
    for (int i=0; i<N; i++) 
    {         
        Y[i] = 0; 
    }  

    //lets time the efficient exclusive scan GPU code  
    cudaEventRecord(start, 0); 
      work_efficient_exc_scan_kernel<<<1,N>>>(X, Y, N);    
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop); 
    cudaEventElapsedTime(&gpu4_elapsed_time_ms, start, stop);

    if(N<10) {
      puts("----------------");
      puts("Results on GPU (exclusive efficient): ");
      puts("X[i]\t| Y[i]");
      puts("--------+-------");

      for(int i=0; i<N; ++i)
        printf("%3d\t| %3d\n", X[i], Y[i]);
    } 

    //output the result
    puts("Performance Results:");
    printf("CPU Time: %3.3f msecs\n", cpu_elapsed_time_ms);
    printf("Inefficient Scans: GPU Time (inclusive scan): %3.3f, GPU (exclusive scan): %3.3f\n",gpu1_elapsed_time_ms, gpu2_elapsed_time_ms);
    printf("Efficient Scans: GPU Time (inclusive scan): %3.3f, GPU (exclusive scan): %3.3f\n",gpu3_elapsed_time_ms, gpu4_elapsed_time_ms);
   

    // free memory on the gpu side
    checkCudaErr( cudaFree( X ) , "cudaFree1");
    checkCudaErr( cudaFree( Y ) , "cudaFree2"); 
		checkCudaErr( cudaDeviceReset(), "cudaDeviceReset");

		return 0;
}

Performance Results:
CPU Time: 0.004 msecs
Inefficient Scans: GPU Time (inclusive scan): 0.298, GPU (exclusive scan): 0.148
Efficient Scans: GPU Time (inclusive scan): 0.141, GPU (exclusive scan): 0.131

